In [27]:
import json
from chromadb.config import Settings
from chromadb.utils import embedding_functions
import chromadb
import os
from dotenv import load_dotenv
import re
load_dotenv()
# show the path of the data generated from the step 1
rule_path = "data/cpp_fix_rules_stratified_ctxt3f5.json"

In [ ]:
# def get_APIs(change):
#     API_list = []
#     # regex to find all APIs in the change
#     API = re.findall(r"\.[a-zA-Z0-9_]+\(", change) 
#     if len(API) > 0:
#         API_list = list(set(API))
#     return API_list

In [ ]:
# def get_APIs(change):
#     """
#     Extracts both C++ member/namespace API calls and C-style function calls from a code change.
#     Returns a list of unique API call substrings.
#     """
#     # Pattern for C++ style: matches calls preceded by ->, . or ::
#     pattern_cpp = r"(?:->|\.|::)[a-zA-Z0-9_]+\("
#     # Pattern for C-style: matches function calls that are not necessarily preceded by any operator.
#     pattern_c = r"\b[a-zA-Z_][a-zA-Z0-9_]*\("

#     # Find matches
#     apis_cpp = re.findall(pattern_cpp, change)
#     apis_c = re.findall(pattern_c, change)
    
#     # Optionally remove the ones already captured by pattern_cpp from pattern_c to avoid duplicates.
#     # This is one simple way to ensure that calls captured with operators are not repeated.
#     apis_c = [api for api in apis_c if not any(api in cpp_api for cpp_api in apis_cpp)]
    
#     # Combine results and remove duplicates
#     all_apis = set(apis_cpp + apis_c)
#     return list(all_apis)

In [28]:
def parse_fix_pattern(response: str) -> str:
    pattern_key = "<pattern>:"
    lower_response = response.lower()
    index = lower_response.find(pattern_key.lower())

    # Case 1: <pattern>: is found
    if index != -1:
        start_idx = index + len(pattern_key)
        extracted = response[start_idx:].strip()
        if extracted.lower() == "na":
            return ""
        return extracted

    # Case 2: <pattern>: is not found
    stripped_response = response.strip()
    if stripped_response.lower() == "na":
        return ""
    return stripped_response

In [ ]:
# def parse_fix_pattern(input_str):

#     if not input_str.lower().strip() == "na": #it misses some cases like "NA" in a long string
#         output_str = input_str
#         # print("fix_pattern: ", output_str)
#     else:
#         # print("error: fix_pattern: not found")
#         output_str = ""
#     return output_str

In [29]:
#chroma db client
# Define persist directory
persist_dir = "/home/akazad/test_tool_perfapim/llmapitool/my_tool_exp/chroma_db"

# Create directory if it doesn't exist
os.makedirs(persist_dir, exist_ok=True)
# after

client = chromadb.PersistentClient(path=persist_dir)


In [30]:
# embedding functions
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=os.environ["OPENAI_API_KEY"],
    model_name="text-embedding-ada-002"
)

In [31]:
# API_misuse_fix_pattern_rules
collection = client.get_or_create_collection(
    "cpp_fix_rules_stratified_ctxt3f5_enbedding", embedding_function=openai_ef)
collection_list = client.list_collections()
print(collection_list)

[Collection(name=cpp_fix_rules_stratified_t3f1_enbedding), Collection(name=cpp_fix_rules_stratified_ctxt3f5_enbedding), Collection(name=cpp_fix_rules_stratified_ctxt3f3_enbedding), Collection(name=cpp_fix_rules_s3_enbedding), Collection(name=cpp_fix_rules_stratified_ctxt3f1_enbedding), Collection(name=cpp_fix_rules_stratified_t3f5_enbedding), Collection(name=cpp_fix_rules_stratified_t3f2_enbedding), Collection(name=cpp_fix_rules_stratified_ctxt2f2_enbedding), Collection(name=cpp_fix_rules_v1_enbedding_975), Collection(name=cpp_fix_rules_stratified_ctxt2f1_enbedding), Collection(name=cpp_fix_rules_stratified_ctxt4f1_enbedding), Collection(name=cpp_fix_rules_s3_t2_enbedding), Collection(name=cpp_fix_rules_stratified_f2_enbedding), Collection(name=cpp_fix_rules_stratified_ctxf1_enbedding), Collection(name=cpp_fix_rules_stratified_f1_enbedding), Collection(name=cpp_fix_rules_stratified_t3f3_enbedding), Collection(name=cpp_fix_rules_stratified_t2f1_enbedding), Collection(name=cpp_fix_rules_

In [32]:
# Load the training data from the file
import json
input_path = "/home/akazad/test_tool_perfapim/perfapim/dataset/stratified_splits/ctx_t3_fold_5_train.jsonl"
data_dict = {}
with open(input_path, encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        try:
            record = json.loads(line)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")
            continue
        data_dict[record["number"]] = record

In [33]:
print(data_dict.keys())

dict_keys([1, 3, 4, 8, 13, 15, 16, 24, 29, 36, 38, 39, 40, 41, 53, 54, 57, 59, 62, 64, 65, 67, 70, 72, 73, 74, 78, 84, 94, 99, 102, 106, 107, 108, 118, 120, 128, 131, 136, 138, 142, 143, 146, 148, 156, 173, 174, 180, 181, 190, 198, 201, 204, 205, 208, 214, 215, 218, 219, 225, 227, 228, 232, 241, 242, 251, 253, 255, 256, 258, 261, 274, 286, 288, 291, 292, 308, 310, 311, 318, 321, 324, 334, 335, 343, 344, 346, 349, 352, 354, 364, 365, 367, 368, 371, 372, 382, 383, 385, 394, 398, 402, 407, 409, 412, 416, 417, 427, 430, 438, 440, 447, 452, 454, 456, 459, 474, 475, 476, 482, 494, 505, 508, 509, 513, 521, 524, 529, 538, 540, 541, 543, 544, 546, 547, 550, 554, 562, 564, 566, 572, 573, 575, 578, 579, 581, 583, 584, 585, 587, 589, 590, 593, 595, 598, 602, 606, 607, 610, 613, 616, 619, 624, 630, 631, 632, 639, 648, 650, 653, 657, 677, 679, 680, 684, 685, 686, 694, 695, 701, 702, 707, 709, 711, 712, 715, 716, 720, 722, 725, 742, 743, 748, 749, 751, 765, 768, 772, 778, 780, 790, 795, 798, 799, 803

In [34]:
print(data_dict[4]['API'])

['::Open(', 'strstr(', '::SetCacheFileDir(']


In [35]:
# this is basically cleaning the data and preparing it for the database
rule_dict = {}
with open(rule_path, encoding="utf-8") as f:
    data = f.readlines()
    data = [line for line in data if line != "\n"]
    data = [json.loads(line) for line in data]
    for line in data:
        change = line["change"]
        n = line["number"]
        APIs = data_dict[n]['API']
        link = data_dict[n]['commit_url_x']
        #APIs = get_APIs(change)
        print(f"N:{n},L:{link},APIs:{APIs}")
        fix_pattern = parse_fix_pattern(line["fix_pattern"]) # returns "" if fix_pattern is "na"
        #filter out the lines that do not have any APIs or fix_pattern
        if len(APIs) > 0 and fix_pattern != "":
            item = {
                "number": line["number"],
                "change": line["change"],
                "APIs": APIs,
                "fix_pattern": fix_pattern,
            }
            rule_dict[line["number"]] = item

data_list = []
for key in rule_dict.keys():
    data_list.append(rule_dict[key]) # convert the dictionary to a list

print(len(data_list))
print(data_list[0])


N:1,L:https://github.com/root-project/root/commit/c7db9a3fc9710a0ba9d4bd17162b5ff1e6eccb27,APIs:['memcpy(', '->Size(', 'mempcpy(']
N:3,L:https://github.com/root-project/root/commit/99de46014c3ced3f6e6e583abd228f4ecc90b833,APIs:['rand(', '->Uniform(']
N:4,L:https://github.com/root-project/root/commit/f6e2ad26ce9abc368e86a6750d750f9ace9481f7,APIs:['::Open(', 'strstr(', '::SetCacheFileDir(']
N:8,L:https://github.com/root-project/root/commit/b4bdc510c1dfe4fa6402f73b4364789b153af35f,APIs:['strlen(', 'strncat(', 'strlcpy(', 'strlcat(', 'strncpy(']
N:13,L:https://github.com/root-project/root/commit/bef0c67f2dff952e6b2e0429261e2be0dc629145,APIs:['Py_DECREF(', 'PyObject_CallMethod(', 'Py_INCREF(']
N:15,L:https://github.com/samanbarghi/uThreads/commit/68e2c0883039201697a4be0b02a2177578d5b8a4,APIs:['free(', 'munmap(']
N:16,L:https://github.com/samhocevar/portable-file-dialogs/commit/0e9410aa57421269d482c6b6136e016c44ab0a3f,APIs:['StringCchCopyW(', '.c_str(', 'lstrcpyW(']
N:24,L:https://github.com

In [36]:
# once prepared, now inject the data into the database
documents = []
ids = []
APIs_list = []
for x in data_list:
    fix_pattn = x["fix_pattern"]
    if fix_pattn == "":
        continue
    print("=====================================")
    print(fix_pattn)
    documents.append(fix_pattn)
    ids.append(str(x["number"])) # number is the id
    APIs_list.append("|".join(x["APIs"])) # APIs is a list of apis
    print(x["APIs"])

print("start db injection")


# 1) Quick sanity check: all three lists must be the same length
assert len(documents) == len(ids) == len(APIs_list), "Length mismatch!"

collection.add(
    documents=documents, # documents is a list of fix patterns, its a list of strings
    ids=ids,
    metadatas = [{"APIs": x} for x in APIs_list]
)
print("finished")

#database preparation done

if the code uses mempcpy unnecessarily when memcpy suffices, replace it with memcpy for improved performance since memcpy is typically optimized for this use case and avoids potential unnecessary complications.
['memcpy(', '->Size(', 'mempcpy(']
if a call to gRandom->Uniform is replaced with a direct use of rand() from the Standard C Library, consider if this change is made to avoid the overhead or dependency of the ROOT Framework for simpler random number generation.
['rand(', '->Uniform(']
if a filename is detected as a URL starting with "http://", utilize the "CACHEREAD" option when opening the file through the TFile API to potentially improve file access performance by caching the contents locally.
['::Open(', 'strstr(', '::SetCacheFileDir(']
if string manipulation is done using strncpy and strncat, replace them with strlcpy and strlcat for improved safety and performance, as strlcpy and strlcat not only copy and concatenate strings but also ensure null-termination and respect buff